In [1]:
%pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/3e/89/cbca600319463a91ae6a46e537d548900ddf2114df66c902344c4fe6bb4c/pandas-2.1.4-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached pandas-2.1.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (18 kB)
  Obtaining dependency information for numpy<2,>=1.23.2 from https://files.pythonhosted.org/packages/2e/54/218ce51bb571a70975f223671b2a86aa951e83abfd2a416a3d540f35115c/numpy-1.26.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached numpy-1.26.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (115 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
Using cached pandas-2.1.4-cp311-cp311-macosx_11_0_arm64.whl (

In [4]:
import pandas as pd

Load raw data from csv files

In [25]:
raw_data_vitalsign = pd.read_csv("../data/physionet.org/files/mimic-iv-ed/2.2/ed/vitalsign.csv")
raw_data_medrecon = pd.read_csv("../data/physionet.org/files/mimic-iv-ed/2.2/ed/medrecon.csv")
raw_data_pyxis = pd.read_csv("../data/physionet.org/files/mimic-iv-ed/2.2/ed/pyxis.csv")

First get vital sequences of length more then 10, then based on those stays we get data for medrecon and pyxis

In [51]:
vitals = raw_data_vitalsign.groupby(["subject_id", "stay_id"]).filter(lambda group: len(group) > 10)
interesting_sequences = vitals.iloc[:, :2] #"subject_id", "stay_id" columns
medrecon = pd.merge(interesting_sequences, raw_data_medrecon, on=["subject_id", "stay_id"], how="left")
pyxis = pd.merge(interesting_sequences, raw_data_pyxis, on=["subject_id", "stay_id"], how="left")
del raw_data_vitalsign, raw_data_medrecon, raw_data_pyxis, interesting_sequences

Join all sequences into one large table

In [56]:
master_data = pd.merge(vitals, medrecon, on=["subject_id", "stay_id", "charttime"], how="outer")
master_data = pd.merge(master_data, pyxis, on=["subject_id", "stay_id", "charttime", "name", "gsn"], how="outer")
del vitals, medrecon, pyxis

In [61]:
master_data.to_csv("../data/master_data.csv")